# Example usage of Stokes Class

In [ ]:
%load_ext autoreload
%autoreload 2

: 

In [ ]:
import numpy as np
import pybbtd.stokes as stokes

In [204]:
# Create Stokes model
X = stokes.Stokes([100, 100], R =3, L=10)

Sufficient condition for uniqueness satisfied
Stokes tensor initialized with dimensions (100, 100, 4) on LL1 mode.


In [205]:
# Create ground truth tensor
np.random.seed(10)
[A0,B0,C0], T0 = X.generate_stokes_tensor()

#Tensor and factors can also be accessed as
T0 = X.tensor
[A0, B0, C0] = X.factors

In [206]:
X.validate_stokes_tensor()

All pixels satisfy the Stokes constraints.


In [214]:
X.draw_Stokes_terms()

NameError: name 'stokes_draw' is not defined